https://qiita.com/_rdtr/items/d3bc1a8d4b7eb375c368

### 関数のネスト
まずinner_functionではローカルxを探すが見つからないので外側の名前空間を探し、outer内で定義されているxを参照します.また#2ではinner()を呼び出していますが, ここで大切なのは innerというのも変数名の1つにすぎず, 解決規則に基づきouter内の名前空間内の定義を探して呼び出されている ということです.

In [2]:
def outer():
    x = "a"
    def inner_function():
        print(x+"hoge") # 1
    inner_function() # 2 
    print(x)
outer()

ahoge
a


In [3]:
def outer():
    x = "a"
    def inner_function():
        x = "hoge"
        print(x+"hoge") # 1
    inner_function() # 2 
    print(x)
outer()

hogehoge
a


In [4]:
# returnにした時

def outer():
    x = "a"
    def inner_function():
        return x+"hoge" # 1
    inner_function() # 2 
    print(x)
outer()

a


In [5]:
def outer():
    x = "a"
    def inner_function():
        return x+"hoge" # 1
    var = inner_function() # 2 
    print(var)
outer()

ahoge


Pythonでは関数自身もまたオブジェクトにすぎません.  
つまり関数を一般的な他の変数と変わりなく、他の関数の引数として与えたり, 関数の戻り値として使うことができます.

In [6]:
issubclass(int, object) # Python内の全てのオブジェクトは同じ共通のクラスを継承して作られている
def foo():
    pass
type(foo)

function

In [7]:
def outer():
    def inner():
        print("Inside inner")
    return inner # 1

foo = outer() #2
foo

<function __main__.outer.<locals>.inner()>

In [8]:
foo()

Inside inner


In [9]:
def outer():
    x = 1
    def inner():
        print(x)
    return inner

foo = outer()
foo.func_closure

AttributeError: 'function' object has no attribute 'func_closure'

In [9]:
def outer(some_func): # パラメータとして関数をとる関数outer()を定義する。
    print(some_func())
    def inner(): # innerは文字列をprintした後、返却する値を取得する。
        print("I am inner function!")
        ret = some_func() #1
        return ret + "hoge"
    return inner

def foo():
    return "a"

In [10]:
decorated = outer(foo) #2
decorated()

a
I am inner function!


'ahoge'

デコレータとは「関数を引数に取り, 引き換えに新たな関数を返すcallable(*)」です.　　


ここでパラメータとしてsome_funcを取るouterという関数を定義しています. そしてouterの中でさらにinnerという内部関数を定義しています.  
innerは文字列をprintした後, #1で返却する値を取得しています. some_funcはouterが呼び出されるごとに異なる値を取りえますが, ここではそれが何であれとりあえず実行(call)し, その実行結果に1を加えた値を返却します.  
最後にouter関数はinner関数そのものを返却します.  

#2ではsome_funcとしてfooを引数にouterを実行した戻り値を変数decoratedに格納しています. fooを実行すると1が返ってきますが, outerを被せたdecoratedではそれに1プラスされて2が返ってきます. 言ってみればdecoratedは, fooのデコレーション版(foo + 何かの処理)といえます.  

実際に有用なデコレータを使う際には, decoratedのように別の変数を用意せずfooそのものを置き換えてしまうことが多いです. つまり下のように.  

In [11]:
foo = outer(foo)

a


In [12]:
foo()

I am inner function!


'ahoge'

In [13]:
foo = outer(foo)
foo()

I am inner function!
ahoge
I am inner function!
I am inner function!


'ahogehoge'

#### なにこれ...なにが嬉しいの...？
とある座標のオブジェクトを保持するライブラリを使っているとします.  
そのオブジェクトはxとyの座標ペアを保持していますが, 残念ながら足し算や引き算など数字の処理機能を持っていません.  
しかし我々はこのライブラリを用いて大量の計算処理をする必要があり, ライブラリのソースを改編することも好ましくない状況だとします.  
どうすれば良いでしょうか？

In [22]:
class Coordinate(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return "Coord: " + str(self.__dict__)
    def add(a, b):
        return Coordinate(a.x + b.x, a.y + b.y)
    def sub(a, b):
        return Coordinate(a.x - b.x, a.y - b.y)

In [29]:
one = Coordinate(100, 200)
two = Coordinate(300, 200)
three = Coordinate(-100, -100)
Coordinate.add(one, two)

Coord: {'x': 400, 'y': 400}

ここでたとえば「扱う座標系は0が下限である必要がある」といったチェック処理が必要だとしたらどうしますか？

In [30]:
def wrapper(func):
    def checker(a, b):
        if a.x < 0 or a.y < 0:
            a = Coordinate(a.x if a.x > 0 else 0, a.y if a.y > 0 else 0)
        if b.x < 0 or b.y < 0:
            b = Coordinate(b.x if b.x > 0 else 0, b.y if b.y > 0 else 0)
        ret = func(a, b)
        if ret.x < 0 or ret.y < 0:
            ret = Coordinate(ret.x if ret.x > 0 else 0, ret.y if ret.y > 0 else 0)
        return ret
    return checker

In [31]:
Coordinate.add = wrapper(Coordinate.add)
Coordinate.sub = wrapper(Coordinate.sub)
Coordinate.sub(one, two)

Coord: {'x': 0, 'y': 0}

In [32]:
Coordinate.add(one, three)

Coord: {'x': 100, 'y': 200}

In [33]:
class Coordinate(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return "Coord: " + str(self.__dict__)
    @wrapper
    def add(a, b):
        return Coordinate(a.x + b.x, a.y + b.y)
    @wrapper
    def sub(a, b):
        return Coordinate(a.x - b.x, a.y - b.y)

関数の引数をログに出力するデコレータを書いてみましょう. 簡略化のためログ出力はstdoutにprintしています

In [34]:
def logger(func):
    def inner(*args, **kwargs): #1
        print("Arguments were: %s, %s" % (args, kwargs))
        return func(*args, **kwargs) #2
    return inner

In [35]:
@logger
def foo1(x, y=1):
    return x * y
@logger
def foo2():
    return 2

In [36]:
foo1(5, 4)

Arguments were: (5, 4), {}


20

In [37]:
foo1(1)

Arguments were: (1,), {}


1

In [38]:
foo2()

Arguments were: (), {}


2

In [45]:
import time
from random import randint


def log(func):
    def wrapper(*args, **kwargs):
        my_arg = ''
        if func.__name__ == 'start_machine':
            my_arg = 'Start Machine\t'
        elif func.__name__ == 'boil_water':
            my_arg = 'Boil Water\t\t'
        elif func.__name__ == 'make_coffee':
            my_arg = 'Make Coffee\t\t'
        elif func.__name__ == 'add_water':
            my_arg = 'Add Water\t\t'
        user_log = '(bcarlier)Running:'
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        with open('machine.log', 'a+') as f:
            f.write(f'{user_log} {my_arg} [ exec-time =  {end_time - start_time:.3f} ms ]\n')
        return result
    return wrapper

class CoffeeMachine():

    water_level = 100

    @log
    def start_machine(self):
        if self.water_level > 20:
            return True
        else:
            print("Please add water!")
            return False

    @log
    def boil_water(self):
        return "boiling..."

    @log
    def make_coffee(self):
        if self.start_machine():
            for _ in range(20):
                time.sleep(0.1)
                self.water_level -= 1
            print(self.boil_water())
            print("Coffee is ready!")

    @log
    def add_water(self, water_level):
        time.sleep(randint(1, 5))
        self.water_level += water_level
        print("Blub blub blub...")

In [46]:
machine = CoffeeMachine()
for i in range(0, 5):
    machine.make_coffee()

machine.make_coffee()
machine.add_water(70)

boiling...
Coffee is ready!
boiling...
Coffee is ready!
boiling...
Coffee is ready!
boiling...
Coffee is ready!
Please add water!
Please add water!
Blub blub blub...


In [47]:
cat machine.log

(bcarlier)Running: Start Machine	 [ exec-time =  0.000 ms ]
(bcarlier)Running: Boil Water		 [ exec-time =  0.000 ms ]
(bcarlier)Running: Make Coffee		 [ exec-time =  2.058 ms ]
(bcarlier)Running: Start Machine	 [ exec-time =  0.000 ms ]
(bcarlier)Running: Boil Water		 [ exec-time =  0.000 ms ]
(bcarlier)Running: Make Coffee		 [ exec-time =  2.072 ms ]
(bcarlier)Running: Start Machine	 [ exec-time =  0.000 ms ]
(bcarlier)Running: Boil Water		 [ exec-time =  0.000 ms ]
(bcarlier)Running: Make Coffee		 [ exec-time =  2.083 ms ]
(bcarlier)Running: Start Machine	 [ exec-time =  0.000 ms ]
(bcarlier)Running: Boil Water		 [ exec-time =  0.000 ms ]
(bcarlier)Running: Make Coffee		 [ exec-time =  2.059 ms ]
(bcarlier)Running: Start Machine	 [ exec-time =  0.000 ms ]
(bcarlier)Running: Make Coffee		 [ exec-time =  0.001 ms ]
(bcarlier)Running: Start Machine	 [ exec-time =  0.000 ms ]
(bcarlier)Running: Make Coffee		 [ exec-time =  0.001 ms ]
(bcarlier)Running: Add Water		 [ exec-time =  2.003 ms

In [ ]:

machine = CoffeeMachine()
for i in range(0, 5):
    machine.make_coffee()

machine.make_coffee()
machine.add_water(70)
